In [1]:
import os
import sys
import torch
import random
import configs
import numpy as np
import transformers
import torch.nn as nn
from PIL import Image
import tensorflow as tf
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

2023-11-28 13:30:31.494794: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 13:30:31.494896: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 13:30:31.494918: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 13:30:31.503659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 13:30:32.631803: W tensorflow/compiler/

In [2]:
configs.set_seed(42)

In [4]:
device = configs.set_device(3)

There are 8 GPU(s) available.
We will use the GPU: NVIDIA A100-PCIE-40GB


In [5]:
from tensorflow.keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()


image_paths = []
labels = []

save_dir = 'cifar10_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for i in range(len(train_images)):
    image_path = os.path.join(save_dir, f"train_image_{i}.jpg")
    tf.keras.preprocessing.image.save_img(image_path, train_images[i])
    image_paths.append(image_path)
    labels.append(train_labels[i][0])


for i in range(len(test_images)):
    image_path = os.path.join(save_dir, f"test_image_{i}.jpg")
    tf.keras.preprocessing.image.save_img(image_path, test_images[i])
    image_paths.append(image_path)
    labels.append(test_labels[i][0])

print(len(train_images), "\n")
print(len(test_images), "\n")
print(len(image_paths))

50000 

10000 

60000


In [6]:
classes = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}

titles = []
for im in tqdm(range(len(image_paths))):
    titles.append(classes[labels[im]])

  0%|          | 0/60000 [00:00<?, ?it/s]

In [7]:
def remove_prefixes(strings):
    prefixes = ['a', 'an', 'the']
    result = []

    for string in strings:
        words = string.split()
        if words[0].lower() in prefixes:
            result.append(' '.join(words[1:]))
        else:
            result.append(string)

    return result

with open("conceptnet_cifar10_filtered_new.txt", "r") as f:
    concepts = f.read().lower().split("\n")
    concepts = remove_prefixes(concepts)

# without concepts

In [8]:
class CustomDataset():
    def __init__(self, list_image_path, list_txt):
        self.image_path = list_image_path
        self.label  = list_txt

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        image_path = self.image_path[idx]
        image = Image.open(image_path)
        label = self.label[idx]
        return {"image_file_path": image_path, "image": image, "labels": label}

dataset = CustomDataset(list_image_path=image_paths, list_txt=labels)

In [18]:
dataset[0]

{'image_file_path': 'cifar10_images/train_image_0.jpg',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=32x32>,
 'labels': 6}

In [5]:
from datasets import load_dataset

dataset = load_dataset("Andron00e/CIFAR10-custom")
dataset = dataset["train"].train_test_split(test_size=0.2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/andreisemenov/.cache/huggingface/datasets/Andron00e___parquet/Andron00e--CIFAR10-custom-8ce32e3d2e963b73/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
from datasets import DatasetDict

val_test = dataset["test"].train_test_split(test_size=0.5)
dataset = DatasetDict({
    "train": dataset["train"],
    "validation": val_test["train"],
    "test": val_test["test"],
})

In [11]:
def process_data(example_batch):
    inputs = processor.image_processor([x for x in example_batch['image']], return_tensors="pt")
    inputs['labels'] = example_batch['labels']
    return inputs

In [12]:
processed_dataset = dataset.with_transform(process_data)

In [13]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [14]:
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

/tmp/ipykernel_4127428/2127246122.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [15]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=10
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import wandb

wandb.login()

wandb: Currently logged in as: semenov-andrei-v. Use `wandb login --relogin` to force relogin


True

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vitforimcls",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=False,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  run_name='cifar10-head',
)

In [20]:
from transformers import CLIPProcessor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model, #ViTForImageClassification
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor.image_processor,
)

In [22]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 4 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
100,0.299500,0.341919,0.910833
200,0.228900,0.248246,0.928833
300,0.181100,0.213924,0.935667
400,0.079700,0.181328,0.946000
500,0.112800,0.174138,0.945167
600,0.086000,0.165894,0.951333
700,0.081500,0.146846,0.954667
800,0.048000,0.139286,0.959167
900,0.021000,0.139900,0.960333
1000,0.027100,0.133400,0.964167


/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 4 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead

***** train metrics *****
  epoch                    =           4.0
  total_flos               = 13857641083GF
  train_loss               =        0.1171
  train_runtime            =    2:24:41.75
  train_samples_per_second =        22.115
  train_steps_per_second   =         0.173


In [24]:
metrics = trainer.evaluate(processed_dataset['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 4 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/andreisemenov/anaconda3/envs/andronserv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9678
  eval_loss               =     0.1199
  eval_runtime            = 0:02:49.73
  eval_samples_per_second =     35.348
  eval_steps_per_second   =      0.554


In [26]:
!pip install huggingface_hub

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="Andron00e/ViTForImageClassification", filename="./vitforimcls/config.json")

In [38]:
import huggingface_hub

huggingface_hub.login()

In [40]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

'https://huggingface.co/Andron00e/vitforimcls/tree/main/'

# try with PEFT

In [48]:
torch.cuda.empty_cache()

In [50]:
!nvidia-smi